In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Представьте, что вы работаете в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения:

1. В первую очередь, его интересует показатель retention. Напишите функцию для его подсчета
Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока)

In [2]:
reg = pd.read_csv('/home/jupyter-n.sobolevskaja-14/shared/problem1-reg_data.csv', sep=';')

In [3]:
reg.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


Проверяем данные на предмет размера, пропусков,типа данных, уникальных значений

In [4]:
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


In [5]:
reg.nunique()

reg_ts    1000000
uid       1000000
dtype: int64

Меняем формат колонки с датой регистрации на "дату", так как задача предполагает расчет retention по дням

In [6]:
reg['reg_ts'] = pd.to_datetime(reg.reg_ts, unit='s')

In [7]:
reg['reg_ts'] =reg['reg_ts'].dt.date
reg.head()

,reg_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,2000-01-13,3
3,2000-05-28,4
4,2000-09-16,5


Считаем количество уникальных пользователей в каждой когорте датасета

In [8]:
cohorts = reg.groupby('reg_ts', as_index=False).agg({'uid':'count'})\
    .rename(columns={'uid': 'cohort_size'})
cohorts.tail()

,reg_ts,cohort_size
5105,2020-09-19,1634
5106,2020-09-20,1636
5107,2020-09-21,1638
5108,2020-09-22,1641
5109,2020-09-23,1048


Делаем предварительный анализ данных из второго датасета и приводим их к нужному нам формату

In [9]:
auth = pd.read_csv('/home/jupyter-n.sobolevskaja-14/shared/problem1-auth_data.csv', sep=';')

In [10]:
auth.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [11]:
auth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


In [12]:
auth.shape

(9601013, 2)

In [13]:
auth.isnull().sum()

auth_ts    0
uid        0
dtype: int64

In [14]:
auth.uid.nunique()

1000000

Видим, что количество уникальных id совпадает с аналогичной колонкой из первого датасета

In [15]:
auth['auth_ts'] = pd.to_datetime(auth.auth_ts, unit='s')
auth['auth_ts'] = auth['auth_ts'].dt.date
auth.head()

,auth_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,1999-07-25,2
3,1999-07-31,2
4,1999-08-05,2


Соединяем датасеты через столбец с уникальными id, проверяем правильность объединения

In [16]:
df = auth.merge(reg, how='left', on=['uid'])
df.shape

(9601013, 3)

Создаем новый столбец: считаем для каждого пользователя количество дней, прошедшее с момента регистрации

In [17]:
df['days_from_reg'] = df.auth_ts - df.reg_ts
df.head()

,auth_ts,uid,reg_ts,days_from_reg
0,1998-11-18,1,1998-11-18,0 days
1,1999-07-22,2,1999-07-22,0 days
2,1999-07-25,2,1999-07-22,3 days
3,1999-07-31,2,1999-07-22,9 days
4,1999-08-05,2,1999-07-22,14 days


Присоединяем к данным датасет с размером каждой когорты

In [18]:
df = df.merge(cohorts, how='left', on=['reg_ts'])
df.shape

(9601013, 5)

Формируем финальный датасет, считая, сколько уникальных id из каждой конкретной когорты заходили в приложение в конкретную дату/через n дней с момента регистрации

In [19]:
df_full = df.groupby(['auth_ts','days_from_reg','reg_ts','cohort_size'], as_index=False)\
    .agg({'uid':'count'})\
    .rename(columns={'auth_ts': 'activity_date','reg_ts' : 'cohort_date'})
df_full.head()

,activity_date,days_from_reg,cohort_date,cohort_size,uid
0,1998-11-18,0 days,1998-11-18,1,1
1,1999-07-22,0 days,1999-07-22,1,1
2,1999-07-25,3 days,1999-07-22,1,1
3,1999-07-31,9 days,1999-07-22,1,1
4,1999-08-05,14 days,1999-07-22,1,1


Добавляем новый столбец Retention. В данном случае, согласно задаче, мы считаем этот показатель по дням от даты регистрации каждой когорты

In [20]:
df_full['retention'] = round(df_full.uid / df_full.cohort_size * 100)
df_full.tail()

,activity_date,days_from_reg,cohort_date,cohort_size,uid,retention
2716223,2020-09-23,4144 days,2009-05-20,2,1,50.0
2716224,2020-09-23,4308 days,2008-12-07,1,1,100.0
2716225,2020-09-23,4332 days,2008-11-13,1,1,100.0
2716226,2020-09-23,4573 days,2008-03-17,1,1,100.0
2716227,2020-09-23,5159 days,2006-08-09,1,1,100.0


Смотрим на получившийся датасет для самопроверки.
Допустим, мы хотим посмотреть на Retention какой-то определенной когорты. Случайно выбираем ее. И видим, что когорта игроков, зарегистрировавшихся в игре 26.01.2018, изначально равнялась 333 уникальным  пользователям, а срок жизни когорты был 971 день с разными показателями значения retention (в основном, 1-2%)

In [21]:
df_full.query('cohort_size == 333')

,activity_date,days_from_reg,cohort_date,cohort_size,uid,retention
1060230,2018-01-26,0 days,2018-01-26,333,333,100.0
1061434,2018-01-27,1 days,2018-01-26,333,2,1.0
1062675,2018-01-28,2 days,2018-01-26,333,15,5.0
1063896,2018-01-29,3 days,2018-01-26,333,16,5.0
1065135,2018-01-30,4 days,2018-01-26,333,18,5.0
...,...,...,...,...,...,...
2706503,2020-09-19,967 days,2018-01-26,333,2,1.0
2708694,2020-09-20,968 days,2018-01-26,333,2,1.0
2710900,2020-09-21,969 days,2018-01-26,333,9,3.0
2713134,2020-09-22,970 days,2018-01-26,333,3,1.0


In [22]:
df_full.query('cohort_size == 333').retention.value_counts()

1.0      548
2.0      336
0.0       39
3.0       17
5.0       11
4.0       10
6.0        2
8.0        1
100.0      1
Name: retention, dtype: int64

Теперь собираем все в функцию, которая будет принимать на вход два изначальных датасета, а отдавать - один обработанный датасет с показателем retention

In [23]:
def retention_func(df_1, df_2):
    df_1['reg_ts'] = pd.to_datetime(df_1.reg_ts, unit='s')
    df_1['reg_ts'] =df_1['reg_ts'].dt.date
    cohorts = df_1.groupby('reg_ts', as_index=False).agg({'uid':'count'})\
    .rename(columns={'uid': 'cohort_size'})
    
    df_2['auth_ts'] = pd.to_datetime(df_2.auth_ts, unit='s')
    df_2['auth_ts'] = df_2['auth_ts'].dt.date
    df = df_2.merge(df_1, how='left', on=['uid'])
    df['days_from_reg'] = df.auth_ts - df.reg_ts
    df = df.merge(cohorts, how='left', on=['reg_ts'])
    
    df_full = df.groupby(['auth_ts','days_from_reg','reg_ts','cohort_size'], as_index=False)\
    .agg({'uid':'count'})\
    .rename(columns={'auth_ts': 'activity_date','reg_ts' : 'cohort_date'})
    df_full['retention'] = round(df_full.uid / df_full.cohort_size * 100)
    return df_full

Переназовем исходные датасеты для проверки работы функции, чтобы она не подтягивала уже обработанные данные из блокнота выше

In [24]:
sign = pd.read_csv('/home/jupyter-n.sobolevskaja-14/shared/problem1-reg_data.csv', sep=';')

In [25]:
active = pd.read_csv('/home/jupyter-n.sobolevskaja-14/shared/problem1-auth_data.csv', sep=';')

In [26]:
retention_func(sign, active)

,activity_date,days_from_reg,cohort_date,cohort_size,uid,retention
0,1998-11-18,0 days,1998-11-18,1,1,100.0
1,1999-07-22,0 days,1999-07-22,1,1,100.0
2,1999-07-25,3 days,1999-07-22,1,1,100.0
3,1999-07-31,9 days,1999-07-22,1,1,100.0
4,1999-08-05,14 days,1999-07-22,1,1,100.0
...,...,...,...,...,...,...
2716223,2020-09-23,4144 days,2009-05-20,2,1,50.0
2716224,2020-09-23,4308 days,2008-12-07,1,1,100.0
2716225,2020-09-23,4332 days,2008-11-13,1,1,100.0
2716226,2020-09-23,4573 days,2008-03-17,1,1,100.0
